In [22]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
M = Vir.verma_module(1/2, 1/2)
d = Vir.basis()
v = M.highest_weight_vector()
m2 = (-4*d[-2] + 3*d[-1]*d[-1])*v
m3 = (-12*d[-2]*d[-1] + 4*d[-1]*d[-1]*d[-1] + 3*d[-3])*v
var('t q')

def L_partition_vector(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def row_to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return sum(row[i]*L_partition_vector(list_of_partition[i], v) for i in range(len(row)))

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^my_len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def partition_to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def my_order(partition):
    polynomial = partition_to_polynomial(partition)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(partition), len(partition) - number_of_ones(partition), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_Q(partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(
        partition,
        [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1], [6, 5, 3, 1]]):
        return True
    if len(partition) > 2:
        for i in range(len(partition) - 2):
            if partition[i] - partition[i + 2] <= 2:
                return True
    if len(partition) > 3:
        for i in range(len(partition) - 3):
            if (partition[i] - partition[i + 3] <= 3
                or (partition[i] - partition[i + 3] <= 4
                    and partition[i + 1] - partition[i + 2] <= 3)):
                return True
    if len(partition) > 4:
        for i in range(len(partition) - 4):
            if (partition[i] - partition[i + 1] == 1 and partition[i + 1] - partition[i + 2] == 2
                and partition[i + 2] - partition[i + 3] == 2
                and partition[i + 3] - partition[i + 4] == 1):
                return True
    return False

def matrix_UVir_m2_m3(n):
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 3)
    matrix_m2 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition2[i], m2))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition3[i], m3))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m3(n)
    pivots = A_n.pivots()
    try:
        return row_to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

def pivots_up_to(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m2_m3(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(n):
    list_of_list_of_partition = []
    pivots = pivots_up_to(n)
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition
             if not partition_contains_any_of_Q(partition)])
    return list_of_list_of_partition

In [5]:
def number_of_ones(a):
    k = 0
    for i in range(len(a)):
        if a[i]==1:
            k = k+1
    return k

def to_dict(element):
    return {tuple(term[0]): term[1] for term in element}

def prt2(a):
    st = ""
    m = number_of_ones(a)
    for i in range(len(a)-m):
        st = st + "x(" + str(a[i]) + ")*"
    st = st + "gen(" + str(m+1) + ")"
    return st

def prt1(u):
    st = ""
    for a in set(u):
        st = st + str(u[a]) + "*" + prt2(a) + "+"
    st = st[:-1]
    return st

def prt(w):
    st = ""
    for i in range(len(w)):
        st = st + prt1(to_dict(w[i])) + ","
    st = st[:-1]
    return st

def subspace_matrix(n): 
    w = []
    q = parts_up_to(n)
    for m in range(n+1):
        a = q[m]
        A = base_subspace(m)
        B = matrix(0, len(a))
        for i in range(m+1):
            b = Partitions1_into_parts(m)[i]
            B2 = matrix(len(b), len(a))
            for j in range(len(b)):
                B2[j, b[j]] = 1
            B = B.stack(B2)    
            C = intersection(B, A)
            w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    ww = [u for u in w if u != {}]
    return ww

def to_partition(a):
    b = []
    max = 0
    while(a[max] == 0):
        max = max + 1
    i = len(a)-2
    while(i!=max-1):
        b = [len(a)-i]*a[i]+b
        i = i-1
    if a[len(a)-1]==3:
        b = b + [1,1]
    if a[len(a)-1]==2:
        b = b + [1]
    return b

def to_polynomial(a,n):
    b = [0]*n
    b[n-1] = number_of_ones(a)
    for i in range(len(a)):
        if a[i]!= 1:
            b[a[i]-2] = b[a[i]-2]+1
    return b        

def avoid_up_to(n):
    # w = subspace_matrix(n)
    w = poisson_submodule_and_differential_ideal_up_to(my_list_of_element, my_list_of_element2, n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    avoid = eval(st)
    for i in range(len(avoid)):
        avoid[i][n-1] = avoid[i][n-1]-1
    return avoid

def weight(element):
    for term in element:
        return sum(list(term[0]))
    
gr_G_M = CombinatorialFreeModule(QQ, Partitions(NonNegativeIntegers()));
B = gr_G_M.basis()

def partial0(partition):
    return sum((partition[k] - 1)*B[Partition(sorted(partition[: k] + [partition[k] + 1] + partition[k + 1:], reverse = True))] for k in range(len(partition)))

def partial(element):
    return sum(term[1]*partial0(list(term[0])) for term in element)

def poisson_bracket0(partition):
    return B[Partition(partition + [1])]

def poisson_bracket(element):
    return sum(term[1]*poisson_bracket0(list(term[0])) for term in element)

def differential_ideal_up_to(list_of_element, n):
    list_of_element2 = []
    for element in list_of_element:
        list_of_element3 = []
        m = n - weight(element)
        if m >= 0:
            list_of_element3 = [element]
            for k in range(m):
                list_of_element3.append(partial(list_of_element3[-1]))
        list_of_element2 += list_of_element3
    return list_of_element2

def poisson_submodule_up_to(list_of_element, n):
    list_of_element2 = []
    for element in list_of_element:
        list_of_element3 = []
        m = n - weight(element)
        if m >= 0:
            list_of_element3 = [element]
            for k in range(m):
                list_of_element3.append(poisson_bracket(list_of_element3[-1]))
        list_of_element2 += list_of_element3
    return list_of_element2

def poisson_submodule_and_differential_ideal_up_to(list_of_element, list_of_element2, n):
    return poisson_submodule_up_to(list_of_element + differential_ideal_up_to(list_of_element2, n), n)

m2 = B[Partition([2])] - 3/4*B[Partition([1, 1])]
m3 = B[Partition([1, 1, 1])]
m5 = B[Partition([3, 1, 1])]
m6 = B[Partition([3, 3])] + 1/3*B[Partition([4, 1, 1])]
m8 = B[Partition([4, 3, 1])]
m9 = B[Partition([4, 4, 1])] + 9/8*B[Partition([5, 3, 1])]
m11 = B[Partition([5, 4, 1, 1])]
m15 = B[Partition([6, 5, 3, 1])]
a = B[Partition([2, 2, 2])]
b = B[Partition([4, 3, 2])] + 1/6*B[Partition([5, 2, 2])]

my_list_of_element = [m2, m3, m5, m6, m8, m9, m11, m15]
my_list_of_element2 = [a, b]

prt(poisson_submodule_and_differential_ideal_up_to(my_list_of_element, my_list_of_element2, 4))

'-3/4*gen(3)+1*x(2)*gen(1),-3/4*gen(4)+1*x(2)*gen(2),1*x(2)*gen(3)+-3/4*gen(5),1*gen(4),1*gen(5)'

In [8]:
def to_polynomial(a,n):
    b = [0]*n
    b[n-1] = number_of_ones(a)
    for i in range(len(a)):
        if a[i]!= 1:
            b[a[i]-2] = b[a[i]-2]+1
    return b        

def p_divides_q(p,q,n):
    for i in range(n):
        if p[i]>q[i]:
            return false
    return true

def a_contains_any_of(a, avoid, n):
    p = to_polynomial(a, n)
    if p[n-1]>2:
        return true
    for i in range(len(avoid)):
        if p[n-1] == avoid[i][n-1] and p_divides_q(avoid[i][:-1], p[:-1], n-1):
            return true
    return false

def base_up_to(n):
    base = [[[]], [[1]], [[1,1]]]
    avoid = avoid_up_to(n)
    # q = parts_up_to(n)
    for i in range(3, n+1):
        add = []
        a = ordered_partitions(i)
        for j in range(len(a)):
            if not a_contains_any_of(a[j], avoid, n):
                add = add + [a[j]]
        base = base + [add]
    return base

In [23]:
base_up_to(24) == basis_up_tov2(24)

True